<a href="https://colab.research.google.com/github/DeepLearningSaeid/Grad/blob/main/Pure_implimentation_SWAG_Numpy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
def X_activation(x):
    return x

def X_activation_derivative(x):
    return np.ones_like(x)

def X_2_activation(x):
    return (x**2) / 8

def X_2_activation_derivative(x):
    return (x / 4)

def X_3_activation(x):
    return (x**3) / 24

def X_3_activation_derivative(x):
    return (x**2) / 8

class NeuralNetwork:
    def __init__(self, input_size, hidden1_size, hidden2_size, hidden3_size, output_size):
        # Define the architecture
        self.input_size = input_size
        self.hidden1_size = hidden1_size
        self.hidden2_size = hidden2_size
        self.hidden3_size = hidden3_size
        self.output_size = output_size

        # Initialize weights and biases
        self.weights = {
            'W1': np.random.randn(input_size, hidden1_size),
            'W2': np.random.randn(input_size, hidden2_size),
            'W3': np.random.randn(input_size, hidden3_size),
            'W4': np.random.randn(hidden1_size + hidden2_size + hidden3_size, output_size)
        }

        self.biases = {
            'b1': np.random.randn(1, hidden1_size),
            'b2': np.random.randn(1, hidden2_size),
            'b3': np.random.randn(1, hidden3_size),
            'b4': np.random.randn(1, output_size)
        }

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def sigmoid_derivative(self, x):
        return x * (1 - x)

    def X_activation(self, x):
        return X_activation(x)

    def X_activation_derivative(self, x):
        return X_activation_derivative(x)

    def X_2_activation(self, x):
        return X_2_activation(x)

    def X_2_activation_derivative(self, x):
        return X_2_activation_derivative(x)

    def X_3_activation(self, x):
        return X_3_activation(x)

    def X_3_activation_derivative(self, x):
        return X_3_activation_derivative(x)

    def feedforward(self, X):
        # Layer 1 (X Activation)
        self.z1 = np.dot(X, self.weights['W1']) + self.biases['b1']
        self.a1 = self.X_activation(self.z1)  # Use X activation for layer

        # Layer 2
        self.z2 = np.dot(X, self.weights['W2']) + self.biases['b2']
        self.a2 = self.X_2_activation(self.z2)

        # Layer 3
        self.z3 = np.dot(X, self.weights['W3']) + self.biases['b3']
        self.a3 = self.X_3_activation(self.z3)

        # Concatenate the outputs of layers 1, 2, and 3
        self.concatenated_output = np.concatenate((self.a1, self.a2, self.a3), axis=1)

        # Layer 4 (Output Layer)
        self.z4 = np.dot(self.concatenated_output, self.weights['W4']) + self.biases['b4']
        self.output = self.X_activation(self.z4)

        return self.output

    def backpropagation(self, X, y, learning_rate):
        # Backpropagation

        # Layer 4 (Output Layer)
        delta4 = 2 * (self.output - y) * self.X_activation_derivative(self.output)
        dW4 = np.dot(self.concatenated_output.T, delta4)
        db4 = np.sum(delta4, axis=0, keepdims=True)

        # Split the delta for the concatenation in layer 4
        delta4_split = np.dot(delta4, self.weights['W4'].T)

        # Separate the deltas for layers 1, 2, and 3
        delta3 = delta4_split[:, -self.hidden3_size:] * self.X_3_activation_derivative(self.a3)

        delta2 = delta4_split[:, -self.hidden3_size-self.hidden2_size:-self.hidden3_size] * self.X_2_activation_derivative(self.a2)
        delta1 = delta4_split[:, :-self.hidden3_size-self.hidden2_size] * self.X_activation_derivative(self.a1)

        # Calculate gradients for layers 1, 2, and 3
        dW3 = np.dot(X.T, delta3)
        db3 = np.sum(delta3, axis=0, keepdims=True)
        dW2 = np.dot(X.T, delta2)
        db2 = np.sum(delta2, axis=0, keepdims=True)
        dW1 = np.dot(X.T, delta1)
        db1 = np.sum(delta1, axis=0, keepdims=True)

        # Update weights and biases
        self.weights['W4'] -= learning_rate * dW4
        self.biases['b4'] -= learning_rate * db4
        self.weights['W3'] -= learning_rate * dW3
        self.biases['b3'] -= learning_rate * db3
        self.weights['W2'] -= learning_rate * dW2
        self.biases['b2'] -= learning_rate * db2
        self.weights['W1'] -= learning_rate * dW1
        self.biases['b1'] -= learning_rate * db1

        return dW4, db4, dW3, db3, dW2, db2, dW1, db1

    def train(self, X, y, learning_rate, epochs):
        for epoch in range(epochs):
            # Forward pass
            self.feedforward(X)

            # Backpropagation
            self.backpropagation(X, y, learning_rate)

            if epoch % 500 == 0:
                loss = np.mean((self.output - y) ** 2)
                print(f'Epoch {epoch}/{epochs}, Loss: {loss:.4f}')

##########################################################
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import time

iris = load_iris()
X = iris.data
y = iris.target
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Add a small epsilon value to avoid exactly zero or one values
epsilon = 1e-4
X = np.clip(X, epsilon, 1 - epsilon)  # Clip values to be in the range (epsilon, 1 - epsilon)

encoder = OneHotEncoder(sparse=False)
y = encoder.fit_transform(y.reshape(-1, 1))

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


learning_rate = 0.00001
epochs = 3000


input_size = 4
hidden1_size = 8
hidden2_size = 8
hidden3_size = 8
output_size = 3
# Create the neural network
nn = NeuralNetwork(input_size, hidden1_size, hidden2_size,hidden3_size, output_size)

# Train the neural network
start_time = time.time()

nn.train(X_train, y_train, learning_rate, epochs)

execution_time = time.time() - start_time

print(f"Training Execution Time: {execution_time:.2f} seconds")
# Evaluate the trained model
def accuracy(y_true, y_pred):
    return np.mean(np.argmax(y_true, axis=1) == np.argmax(y_pred, axis=1))

y_pred = nn.feedforward(X_test)
acc = accuracy(y_test, y_pred)
print(f'Test Accuracy: {acc * 100:.2f}%')

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Epoch 0/3000, Loss: 18.2705
Epoch 500/3000, Loss: 0.2485
Epoch 1000/3000, Loss: 0.1756
Epoch 1500/3000, Loss: 0.1599
Epoch 2000/3000, Loss: 0.1498
Epoch 2500/3000, Loss: 0.1420
Training Execution Time: 1.45 seconds
Test Accuracy: 73.33%


In [ ]:
def X_activation(x):
    return x

def X_activation_derivative(x):
    return np.ones_like(x)

def X_2_activation(x):
    return (x**2) / 8

def X_2_activation_derivative(x):
    return (x / 4)

def X_3_activation(x):
    return (x**3) / 24

def X_3_activation_derivative(x):
    return (x**2) / 8

class NeuralNetwork:
    def __init__(self, input_size, hidden1_size, hidden2_size, hidden3_size, output_size):
        # Define the architecture
        self.input_size = input_size
        self.hidden1_size = hidden1_size
        self.hidden2_size = hidden2_size
        self.hidden3_size = hidden3_size
        self.output_size = output_size

        # Initialize weights and biases
        self.weights = {
            'W1': np.random.randn(input_size, hidden1_size),
            'W2': np.random.randn(input_size, hidden2_size),
            'W3': np.random.randn(input_size, hidden3_size),
            'W4': np.random.randn(hidden1_size + hidden2_size + hidden3_size, output_size)
        }

        self.biases = {
            'b1': np.random.randn(1, hidden1_size),
            'b2': np.random.randn(1, hidden2_size),
            'b3': np.random.randn(1, hidden3_size),
            'b4': np.random.randn(1, output_size)
        }

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def sigmoid_derivative(self, x):
        return x * (1 - x)

    def X_activation(self, x):
        return X_activation(x)

    def X_activation_derivative(self, x):
        return X_activation_derivative(x)

    def X_2_activation(self, x):
        return X_2_activation(x)

    def X_2_activation_derivative(self, x):
        return X_2_activation_derivative(x)

    def X_3_activation(self, x):
        return X_3_activation(x)

    def X_3_activation_derivative(self, x):
        return X_3_activation_derivative(x)

    def feedforward(self, X):
        # Layer 1 (X Activation)
        self.z1 = np.dot(X, self.weights['W1']) + self.biases['b1']
        self.a1 = self.X_activation(self.z1)  # Use X activation for layer

        # Layer 2
        self.z2 = np.dot(X, self.weights['W2']) + self.biases['b2']
        self.a2 = self.X_2_activation(self.z2)

        # Layer 3
        self.z3 = np.dot(X, self.weights['W3']) + self.biases['b3']
        self.a3 = self.X_3_activation(self.z3)

        # Concatenate the outputs of layers 1, 2, and 3
        self.concatenated_output = np.concatenate((self.a1, self.a2, self.a3), axis=1)

        # Layer 4 (Output Layer)
        self.z4 = np.dot(self.concatenated_output, self.weights['W4']) + self.biases['b4']
        self.output = self.X_activation(self.z4)

        return self.output

    def backpropagation(self, X, y, learning_rate):
        # Backpropagation

        # Layer 4 (Output Layer)
        delta4 = 2 * (self.output - y) * self.X_activation_derivative(self.output)
        dW4 = np.dot(self.concatenated_output.T, delta4)
        db4 = np.sum(delta4, axis=0, keepdims=True)

        # Split the delta for the concatenation in layer 4
        delta4_split = np.dot(delta4, self.weights['W4'].T)

        # Separate the deltas for layers 1, 2, and 3
        delta3 = delta4_split[:, -self.hidden3_size:] * self.X_3_activation_derivative(self.a3)

        delta2 = delta4_split[:, -self.hidden3_size-self.hidden2_size:-self.hidden3_size] * self.X_2_activation_derivative(self.a2)
        delta1 = delta4_split[:, :-self.hidden3_size-self.hidden2_size] * self.X_activation_derivative(self.a1)

        # Calculate gradients for layers 1, 2, and 3
        dW3 = np.dot(X.T, delta3)
        db3 = np.sum(delta3, axis=0, keepdims=True)
        dW2 = np.dot(X.T, delta2)
        db2 = np.sum(delta2, axis=0, keepdims=True)
        dW1 = np.dot(X.T, delta1)
        db1 = np.sum(delta1, axis=0, keepdims=True)

        # Update weights and biases
        self.weights['W4'] -= learning_rate * dW4
        self.biases['b4'] -= learning_rate * db4
        self.weights['W3'] -= learning_rate * dW3
        self.biases['b3'] -= learning_rate * db3
        self.weights['W2'] -= learning_rate * dW2
        self.biases['b2'] -= learning_rate * db2
        self.weights['W1'] -= learning_rate * dW1
        self.biases['b1'] -= learning_rate * db1

        return dW4, db4, dW3, db3, dW2, db2, dW1, db1

    def train(self, X, y, learning_rate, epochs):
        batch_size = 10
        n_batches = len(X) // batch_size

        for epoch in range(epochs):
            epoch_loss = 0  # Reset epoch loss

            for i in range(0, len(X), batch_size):
                X_batch = X[i:i + batch_size]
                y_batch = y[i:i + batch_size]

                # Forward pass
                self.feedforward(X_batch)

                # Backpropagation
                self.backpropagation(X_batch, y_batch, learning_rate)

                # Calculate batch loss and add it to epoch loss
                batch_loss = np.mean((self.output - y_batch) ** 2)
                epoch_loss += batch_loss

            # Calculate average epoch loss
            epoch_loss /= n_batches

            if epoch % 500 == 0:
                print(f'Epoch {epoch}/{epochs}, Loss: {epoch_loss:.4f}')

##########################################################
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import time

iris = load_iris()
X = iris.data
y = iris.target
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Add a small epsilon value to avoid exactly zero or one values
epsilon = 1e-4
X = np.clip(X, epsilon, 1 - epsilon)  # Clip values to be in the range (epsilon, 1 - epsilon)

encoder = OneHotEncoder(sparse=False)
y = encoder.fit_transform(y.reshape(-1, 1))

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


learning_rate = 0.0001
epochs = 100


input_size = 4
hidden1_size = 8
hidden2_size = 8
hidden3_size = 8
output_size = 3
# Create the neural network
nn = NeuralNetwork(input_size, hidden1_size, hidden2_size,hidden3_size, output_size)

# Train the neural network
start_time = time.time()

nn.train(X_train, y_train, learning_rate, epochs)

execution_time = time.time() - start_time

print(f"Training Execution Time: {execution_time:.2f} seconds")
# Evaluate the trained model
def accuracy(y_true, y_pred):
    return np.mean(np.argmax(y_true, axis=1) == np.argmax(y_pred, axis=1))

y_pred = nn.feedforward(X_test)
acc = accuracy(y_test, y_pred)
print(f'Test Accuracy: {acc * 100:.2f}%')

Epoch 0/100, Loss: 11.0058
Training Execution Time: 0.19 seconds
Test Accuracy: 70.00%


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [ ]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Activation functions
def identity(x):
    return x

def square(x):
    return np.power(x, 2)

def identity_derivative(x):
    return np.ones_like(x)

def square_derivative(x):
    return 2 * x

# Initialize network parameters
def initialize_parameters(input_size, hidden_size1, hidden_size2, output_size):
    return {
        'W1': np.random.randn(input_size, hidden_size1) * 0.1,
        'b1': np.zeros((1, hidden_size1)),
        'W2': np.random.randn(input_size, hidden_size2) * 0.1,
        'b2': np.zeros((1, hidden_size2)),
        'W3': np.random.randn(hidden_size1 + hidden_size2, output_size) * 0.1,
        'b3': np.zeros((1, output_size))
    }

# Forward pass
def forward_pass(X, params):
    Z1 = np.dot(X, params['W1']) + params['b1']
    A1 = identity(Z1)

    Z2 = np.dot(X, params['W2']) + params['b2']
    A2 = square(Z2)

    concatenated = np.concatenate((A1, A2), axis=1)

    Z3 = np.dot(concatenated, params['W3']) + params['b3']
    A3 = square(Z3)

    return A3, (X, Z1, A1, Z2, A2, Z3, A3, concatenated)

# Compute loss (Mean Squared Error)
def compute_loss(y_true, y_pred):
    return np.mean(np.square(y_true - y_pred))

# Backward pass
def backward_pass(y_true, cache, params):
    X, Z1, A1, Z2, A2, Z3, A3, concatenated = cache

    dA3 = 2 * (A3 - y_true)
    dZ3 = dA3 * square_derivative(Z3)
    dW3 = np.dot(concatenated.T, dZ3)
    db3 = np.sum(dZ3, axis=0, keepdims=True)

    d_concatenated = np.dot(dZ3, params['W3'].T)
    dA1, dA2 = d_concatenated[:, :hidden_size1], d_concatenated[:, hidden_size1:]

    dZ1 = dA1 * identity_derivative(Z1)
    dW1 = np.dot(X.T, dZ1)
    db1 = np.sum(dZ1, axis=0, keepdims=True)

    dZ2 = dA2 * square_derivative(Z2)
    dW2 = np.dot(X.T, dZ2)
    db2 = np.sum(dZ2, axis=0, keepdims=True)

    grads = {'dW1': dW1, 'db1': db1, 'dW2': dW2, 'db2': db2, 'dW3': dW3, 'db3': db3}
    return grads

# Update network parameters
def update_parameters(params, grads, learning_rate):
    for key in params.keys():
        params[key] -= learning_rate * grads['d' + key]
    return params

# Load and preprocess the Iris dataset
iris = load_iris()
X, y = iris.data, iris.target
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
Y_onehot = np.eye(3)[y]  # Convert to one-hot encoding

# Split the dataset
X_train, X_test, Y_train, Y_test = train_test_split(X_scaled, Y_onehot, test_size=0.2, random_state=42)

# Network architecture
input_size = X_train.shape[1]
hidden_size1 = 5
hidden_size2 = 5
output_size = 3

# Initialize parameters
params = initialize_parameters(input_size, hidden_size1, hidden_size2, output_size)

# Training settings
epochs = 200
learning_rate = 0.001

# Training loop
for epoch in range(epochs):
    output, cache = forward_pass(X_train, params)
    loss = compute_loss(Y_train, output)
    grads = backward_pass(Y_train, cache, params)
    params = update_parameters(params, grads, learning_rate)

    if epoch % 20 == 0:
        print(f"Epoch {epoch}, Loss: {loss:.4f}")

# Evaluate the model
output_test, _ = forward_pass(X_test, params)
test_loss = compute_loss(Y_test, output_test)
predictions = np.argmax(output_test, axis=1)
# Convert predictions to one-hot encoded format
predictions_onehot = np.eye(output_size)[predictions]

# Compute accuracy using one-hot encoded predictions
accuracy = np.mean(np.all(predictions_onehot == Y_test, axis=1))

print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")


Epoch 0, Loss: 0.3331
Epoch 20, Loss: 0.2046
Epoch 40, Loss: 0.0684
Epoch 60, Loss: 0.0364
Epoch 80, Loss: 0.0379
Epoch 100, Loss: 0.0347
Epoch 120, Loss: 0.0330
Epoch 140, Loss: 0.0318
Epoch 160, Loss: 0.0309
Epoch 180, Loss: 0.0303
Test Loss: 0.0728
Test Accuracy: 0.9667


In [50]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Activation functions
def identity(x):
    return x

def square(x):
    return np.power(x, 2)

def identity_derivative(x):
    return np.ones_like(x)

def square_derivative(x):
    return 2 * x

# Initialize network parameters
def initialize_parameters(input_size, hidden_size1, hidden_size2, hidden_size3, output_size):
    return {
        'W1': np.random.randn(input_size, hidden_size1) * 0.1,
        'b1': np.zeros((1, hidden_size1)),
        'W2': np.random.randn(input_size, hidden_size2) * 0.1,
        'b2': np.zeros((1, hidden_size2)),
        'W3': np.random.randn(hidden_size1 + hidden_size2, hidden_size3) * 0.1,
        'b3': np.zeros((1, hidden_size3)),
        'W4': np.random.randn(hidden_size3, output_size) * 0.1,
        'b4': np.zeros((1, output_size))
    }

# Forward pass
def forward_pass(X, params):
    Z1 = np.dot(X, params['W1']) + params['b1']
    A1 = identity(Z1)

    Z2 = np.dot(X, params['W2']) + params['b2']
    A2 = square(Z2)

    concatenated = np.concatenate((A1, A2), axis=1)

    Z3 = np.dot(concatenated, params['W3']) + params['b3']
    A3 = square(Z3)


    Z4 = np.dot(A3, params['W4']) + params['b4']
    A4 = identity(Z4)  # Linear activation

    return A4, (X, Z1, A1, Z2, A2, Z3, A3, Z4, A4, concatenated)

# Compute loss (Mean Squared Error)
def compute_loss(y_true, y_pred):
    return np.mean(np.square(y_true - y_pred))

# Backward pass
def backward_pass(y_true, cache, params):
    X, Z1, A1, Z2, A2, Z3, A3, Z4, A4, concatenated = cache

    dA4 = 2 * (A4 - y_true)
    dZ4 = dA4 * identity_derivative(Z4)  # Derivative of linear activation is 1
    dW4 = np.dot(A3.T, dZ4)
    db4 = np.sum(dZ4, axis=0, keepdims=True)

    dA3 = np.dot(dZ4, params['W4'].T)
    dZ3 = dA3 * square_derivative(Z3)
    dW3 = np.dot(concatenated.T, dZ3)
    db3 = np.sum(dZ3, axis=0, keepdims=True)

    d_concatenated = np.dot(dZ3, params['W3'].T)
    dA1, dA2 = d_concatenated[:, :hidden_size1], d_concatenated[:, hidden_size1:]

    dZ1 = dA1 * identity_derivative(Z1)
    dW1 = np.dot(X.T, dZ1)
    db1 = np.sum(dZ1, axis=0, keepdims=True)

    dZ2 = dA2 * square_derivative(Z2)
    dW2 = np.dot(X.T, dZ2)
    db2 = np.sum(dZ2, axis=0, keepdims=True)

    grads = {'dW1': dW1, 'db1': db1, 'dW2': dW2, 'db2': db2, 'dW3': dW3, 'db3': db3, 'dW4': dW4, 'db4': db4}
    return grads

# Update network parameters
def update_parameters(params, grads, learning_rate):
    for key in params.keys():
        params[key] -= learning_rate * grads['d' + key]
    return params

# Load and preprocess the Iris dataset
iris = load_iris()
X, y = iris.data, iris.target
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
Y_onehot = np.eye(3)[y]  # Convert to one-hot encoding

# Split the dataset
X_train, X_test, Y_train, Y_test = train_test_split(X_scaled, Y_onehot, test_size=0.2, random_state=42)

# Network architecture
input_size = X_train.shape[1]
hidden_size1 = 5
hidden_size2 = 5
hidden_size3 = 5
output_size = 3

# Initialize parameters
params = initialize_parameters(input_size, hidden_size1, hidden_size2, hidden_size3, output_size)

# Training settings
epochs = 200
learning_rate = 0.001

# Training loop
for epoch in range(epochs):
    output, cache = forward_pass(X_train, params)
    loss = compute_loss(Y_train, output)
    grads = backward_pass(Y_train, cache, params)
    params = update_parameters(params, grads, learning_rate)

    if epoch % 20 == 0:
        print(f"Epoch {epoch}, Loss: {loss:.4f}")

# Evaluate the model
output_test, _ = forward_pass(X_test, params)
test_loss = compute_loss(Y_test, output_test)
predictions = np.argmax(output_test, axis=1)
# Convert predictions to one-hot encoded format
predictions_onehot = np.eye(output_size)[predictions]

# Compute accuracy using one-hot encoded predictions
accuracy = np.mean(np.all(predictions_onehot == Y_test, axis=1))

print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")


Epoch 0, Loss: 0.3337
Epoch 20, Loss: 0.2220
Epoch 40, Loss: 0.2210
Epoch 60, Loss: 0.2001
Epoch 80, Loss: 0.1148
Epoch 100, Loss: 0.1125
Epoch 120, Loss: 0.1042
Epoch 140, Loss: 0.0653
Epoch 160, Loss: 0.0712
Epoch 180, Loss: 0.0715
Test Loss: 0.0688
Test Accuracy: 0.9333


In [52]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Activation functions
def identity(x):
    return x

def square(x):
    return np.power(x, 2)

def identity_derivative(x):
    return np.ones_like(x)

def square_derivative(x):
    return 2 * x

# Initialize network parameters
def initialize_parameters(input_size, hidden_size1, hidden_size2, hidden_size3, output_size):
    return {
        'W1': np.random.randn(input_size, hidden_size1) * 0.1,
        'b1': np.zeros((1, hidden_size1)),
        'W2': np.random.randn(input_size, hidden_size2) * 0.1,
        'b2': np.zeros((1, hidden_size2)),
        'W3': np.random.randn(hidden_size1 + hidden_size2, hidden_size3) * 0.1,
        'b3': np.zeros((1, hidden_size3)),
        'W4': np.random.randn(hidden_size3, output_size) * 0.1,  # Corrected output size
        'b4': np.zeros((1, output_size))
    }

# Forward pass
def forward_pass(X, params):
    Z1 = np.dot(X, params['W1']) + params['b1']
    A1 = identity(Z1)

    Z2 = np.dot(X, params['W2']) + params['b2']
    A2 = square(Z2)

    concatenated = np.concatenate((A1, A2), axis=1)

    Z3 = np.dot(concatenated, params['W3']) + params['b3']
    A3 = square(Z3)

    concatenated_A3 = np.concatenate((A3, concatenated), axis=1)  # Concatenate A3 and concatenated

    Z4 = np.dot(concatenated_A3, params['W4']) + params['b4']
    A4 = identity(Z4)  # Linear activation

    return A4, (X, Z1, A1, Z2, A2, Z3, A3, concatenated_A3, Z4, A4, concatenated)

# Compute loss (Mean Squared Error)
def compute_loss(y_true, y_pred):
    return np.mean(np.square(y_true - y_pred))

# Backward pass
def backward_pass(y_true, cache, params):
    X, Z1, A1, Z2, A2, Z3, A3, concatenated_A3, Z4, A4, concatenated = cache

    dA4 = 2 * (A4 - y_true)
    dZ4 = dA4 * identity_derivative(Z4)  # Derivative of linear activation is 1
    dW4 = np.dot(concatenated_A3.T, dZ4)  # Update using concatenated A3
    db4 = np.sum(dZ4, axis=0, keepdims=True)

    d_concatenated_A3 = np.dot(dZ4, params['W4'].T)
    dA3 = d_concatenated_A3[:, :hidden_size3]
    d_concatenated = d_concatenated_A3[:, hidden_size3:]

    dZ3 = dA3 * square_derivative(Z3)
    dW3 = np.dot(concatenated.T, dZ3)
    db3 = np.sum(dZ3, axis=0, keepdims=True)

    d_concatenated_Z2 = dA2 * square_derivative(Z2)
    dA1, dA2 = d_concatenated[:, :hidden_size1], d_concatenated[:, hidden_size1:]

    dZ1 = dA1 * identity_derivative(Z1)
    dW1 = np.dot(X.T, dZ1)
    db1 = np.sum(dZ1, axis=0, keepdims=True)

    dZ2 = d_concatenated_Z2
    dW2 = np.dot(X.T, dZ2)
    db2 = np.sum(dZ2, axis=0, keepdims=True)

    grads = {'dW1': dW1, 'db1': db1, 'dW2': dW2, 'db2': db2, 'dW3': dW3, 'db3': db3, 'dW4': dW4, 'db4': db4}
    return grads

# Update network parameters
def update_parameters(params, grads, learning_rate):
    for key in params.keys():
        params[key] -= learning_rate * grads['d' + key]
    return params

# Load and preprocess the Iris dataset
iris = load_iris()
X, y = iris.data, iris.target
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
Y_onehot = np.eye(3)[y]  # Convert to one-hot encoding

# Split the dataset
X_train, X_test, Y_train, Y_test = train_test_split(X_scaled, Y_onehot, test_size=0.2, random_state=42)

# Network architecture
input_size = X_train.shape[1]
hidden_size1 = 5
hidden_size2 = 5
hidden_size3 = 5
output_size = 3

# Initialize parameters
params = initialize_parameters(input_size, hidden_size1, hidden_size2, hidden_size3, output_size)

# Training settings
epochs = 200
learning_rate = 0.001

# Training loop
for epoch in range(epochs):
    output, cache = forward_pass(X_train, params)
    loss = compute_loss(Y_train, output)
    grads = backward_pass(Y_train, cache, params)
    params = update_parameters(params, grads, learning_rate)

    if epoch % 20 == 0:
        print(f"Epoch {epoch}, Loss: {loss:.4f}")

# Evaluate the model
output_test, _ = forward_pass(X_test, params)
test_loss = compute_loss(Y_test, output_test)
predictions = np.argmax(output_test, axis=1)
# Convert predictions to one-hot encoded format
predictions_onehot = np.eye(output_size)[predictions]

# Compute accuracy using one-hot encoded predictions
accuracy = np.mean(np.all(predictions_onehot == Y_test, axis=1))

print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")


ValueError: shapes (120,15) and (5,3) not aligned: 15 (dim 1) != 5 (dim 0)

In [57]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Activation functions
def identity(x):
    return x

def square(x):
    return np.power(x, 2)

def identity_derivative(x):
    return np.ones_like(x)

def square_derivative(x):
    return 2 * x

# Initialize network parameters
def initialize_parameters(input_size, hidden_size1, hidden_size2, hidden_size3, output_size):
    return {
        'W1': np.random.randn(input_size, hidden_size1) * 0.1,
        'b1': np.zeros((1, hidden_size1)),
        'W2': np.random.randn(input_size, hidden_size2) * 0.1,
        'b2': np.zeros((1, hidden_size2)),
        'W3': np.random.randn(hidden_size1 + hidden_size2, hidden_size3) * 0.1,
        'b3': np.zeros((1, hidden_size3)),
        'W4': np.random.randn(hidden_size3 + hidden_size1 + hidden_size2, output_size) * 0.1,
        'b4': np.zeros((1, output_size))
    }

# Forward pass
def forward_pass(X, params):
    Z1 = np.dot(X, params['W1']) + params['b1']
    A1 = identity(Z1)

    Z2 = np.dot(X, params['W2']) + params['b2']
    A2 = square(Z2)

    concatenated = np.concatenate((A1, A2), axis=1)

    Z3 = np.dot(concatenated, params['W3']) + params['b3']
    A3 = square(Z3)

    concatenated_A3 = np.concatenate((A3, concatenated), axis=1)

    Z4 = np.dot(concatenated_A3, params['W4']) + params['b4']
    A4 = identity(Z4)  # Linear activation

    return A4, (X, Z1, A1, Z2, A2, Z3, A3, concatenated_A3, Z4, A4, concatenated)

# Compute loss (Mean Squared Error)
def compute_loss(y_true, y_pred):
    return np.mean(np.square(y_true - y_pred))

# Backward pass
def backward_pass(y_true, cache, params):
    X, Z1, A1, Z2, A2, Z3, A3, concatenated_A3, Z4, A4, concatenated = cache

    dA4 = 2 * (A4 - y_true)
    dZ4 = dA4 * identity_derivative(Z4)  # Derivative of linear activation is 1
    dW4 = np.dot(concatenated_A3.T, dZ4)
    db4 = np.sum(dZ4, axis=0, keepdims=True)

    d_concatenated_A3 = np.dot(dZ4, params['W4'].T)
    dA3 = d_concatenated_A3[:, :hidden_size3]
    d_concatenated = d_concatenated_A3[:, hidden_size3:]

    dZ3 = dA3 * square_derivative(Z3)
    dW3 = np.dot(concatenated.T, dZ3)
    db3 = np.sum(dZ3, axis=0, keepdims=True)

    d_concatenated_Z2 = dA2 * square_derivative(Z2)
    dA1, dA2 = d_concatenated[:, :hidden_size1], d_concatenated[:, hidden_size1:]

    dZ1 = dA1 * identity_derivative(Z1)
    dW1 = np.dot(X.T, dZ1)
    db1 = np.sum(dZ1, axis=0, keepdims=True)

    dZ2 = d_concatenated_Z2[:, hidden_size1:]  # Update dZ2 calculation
    dW2 = np.dot(X.T, dZ2)
    db2 = np.sum(dZ2, axis=0, keepdims=True)

    grads = {'dW1': dW1, 'db1': db1, 'dW2': dW2, 'db2': db2, 'dW3': dW3, 'db3': db3, 'dW4': dW4, 'db4': db4}
    return grads

# Update network parameters
def update_parameters(params, grads, learning_rate):
    for key in params.keys():
        params[key] -= learning_rate * grads['d' + key]
    return params

# Load and preprocess the Iris dataset
iris = load_iris()
X, y = iris.data, iris.target
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
Y_onehot = np.eye(3)[y]

# Split the dataset
X_train, X_test, Y_train, Y_test = train_test_split(X_scaled, Y_onehot, test_size=0.2, random_state=42)

# Network architecture
input_size = X_train.shape[1]
hidden_size1 = 5
hidden_size2 = 5
hidden_size3 = 5
output_size = 3

# Initialize parameters
params = initialize_parameters(input_size, hidden_size1, hidden_size2, hidden_size3, output_size)

# Training settings
epochs = 200
learning_rate = 0.001

# Training loop
for epoch in range(epochs):
    output, cache = forward_pass(X_train, params)
    loss = compute_loss(Y_train, output)
    grads = backward_pass(Y_train, cache, params)
    params = update_parameters(params, grads, learning_rate)

    if epoch % 20 == 0:
        print(f"Epoch {epoch}, Loss: {loss:.4f}")

# Evaluate the model
output_test, _ = forward_pass(X_test, params)
test_loss = compute_loss(Y_test, output_test)
predictions = np.argmax(output_test, axis=1)
predictions_onehot = np.eye(output_size)[predictions]
accuracy = np.mean(np.all(predictions_onehot == Y_test, axis=1))

print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")


UnboundLocalError: local variable 'dA2' referenced before assignment

In [58]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Activation functions
def identity(x):
    return x

def square(x):
    return np.power(x, 2)/4

def square_(x):
    return np.power(x, 2)/24

def identity_derivative(x):
    return np.ones_like(x)

def square_derivative(x):
    return 2 * x

# Initialize network parameters
def initialize_parameters(input_size, hidden_size1, hidden_size2, hidden_size3, output_size):
    return {
        'W1': np.random.randn(input_size, hidden_size1) * 0.1,
        'b1': np.zeros((1, hidden_size1)),
        'W2': np.random.randn(input_size, hidden_size2) * 0.1,
        'b2': np.zeros((1, hidden_size2)),
        'W3': np.random.randn(hidden_size1 + hidden_size2, hidden_size3) * 0.1,
        'b3': np.zeros((1, hidden_size3)),
        'W4': np.random.randn(hidden_size3 + hidden_size1 + hidden_size2, output_size) * 0.1,
        'b4': np.zeros((1, output_size))
    }

# Forward pass
def forward_pass(X, params):
    Z1 = np.dot(X, params['W1']) + params['b1']
    A1 = identity(Z1)

    Z2 = np.dot(X, params['W2']) + params['b2']
    A2 = square(Z2)

    concatenated = np.concatenate((A1, A2), axis=1)

    Z3 = np.dot(concatenated, params['W3']) + params['b3']
    A3 = square_(Z3)

    concatenated_A3 = np.concatenate((A3, concatenated), axis=1)

    Z4 = np.dot(concatenated_A3, params['W4']) + params['b4']
    A4 = identity(Z4)  # Linear activation

    return A4, (X, Z1, A1, Z2, A2, Z3, A3, concatenated_A3, Z4, A4, concatenated)

# Compute loss (Mean Squared Error)
def compute_loss(y_true, y_pred):
    return np.mean(np.square(y_true - y_pred))

# Backward pass
def backward_pass(y_true, cache, params):
    X, Z1, A1, Z2, A2, Z3, A3, concatenated_A3, Z4, A4, concatenated = cache

    dA4 = 2 * (A4 - y_true)
    dZ4 = dA4 * identity_derivative(Z4)  # Derivative of linear activation is 1
    dW4 = np.dot(concatenated_A3.T, dZ4)
    db4 = np.sum(dZ4, axis=0, keepdims=True)

    d_concatenated_A3 = np.dot(dZ4, params['W4'].T)
    dA3 = d_concatenated_A3[:, :hidden_size3]
    d_concatenated = d_concatenated_A3[:, hidden_size3:]

    dZ3 = dA3 * square_derivative(Z3)
    dW3 = np.dot(concatenated.T, dZ3)
    db3 = np.sum(dZ3, axis=0, keepdims=True)

    dA2 = d_concatenated[:, hidden_size1:]  # Corrected dA2 calculation
    d_concatenated_Z2 = dA2 * square_derivative(Z2)

    dA1 = d_concatenated[:, :hidden_size1]
    dZ1 = dA1 * identity_derivative(Z1)
    dW1 = np.dot(X.T, dZ1)
    db1 = np.sum(dZ1, axis=0, keepdims=True)

    dZ2 = d_concatenated_Z2
    dW2 = np.dot(X.T, dZ2)
    db2 = np.sum(dZ2, axis=0, keepdims=True)

    grads = {'dW1': dW1, 'db1': db1, 'dW2': dW2, 'db2': db2, 'dW3': dW3, 'db3': db3, 'dW4': dW4, 'db4': db4}
    return grads

# Update network parameters
def update_parameters(params, grads, learning_rate):
    for key in params.keys():
        params[key] -= learning_rate * grads['d' + key]
    return params

# Load and preprocess the Iris dataset
iris = load_iris()
X, y = iris.data, iris.target
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
Y_onehot = np.eye(3)[y]

# Split the dataset
X_train, X_test, Y_train, Y_test = train_test_split(X_scaled, Y_onehot, test_size=0.2, random_state=42)

# Network architecture
input_size = X_train.shape[1]
hidden_size1 = 5
hidden_size2 = 5
hidden_size3 = 5
output_size = 3

# Initialize parameters
params = initialize_parameters(input_size, hidden_size1, hidden_size2, hidden_size3, output_size)

# Training settings
epochs = 200
learning_rate = 0.001

# Training loop
for epoch in range(epochs):
    output, cache = forward_pass(X_train, params)
    loss = compute_loss(Y_train, output)
    grads = backward_pass(Y_train, cache, params)
    params = update_parameters(params, grads, learning_rate)

    if epoch % 20 == 0:
        print(f"Epoch {epoch}, Loss: {loss:.4f}")

# Evaluate the model
output_test, _ = forward_pass(X_test, params)
test_loss = compute_loss(Y_test, output_test)
predictions = np.argmax(output_test, axis=1)
predictions_onehot = np.eye(output_size)[predictions]
accuracy = np.mean(np.all(predictions_onehot == Y_test, axis=1))

print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")


Epoch 0, Loss: 0.3355
Epoch 20, Loss: 0.1032
Epoch 40, Loss: 0.0946
Epoch 60, Loss: 0.0688
Epoch 80, Loss: 0.0483
Epoch 100, Loss: 0.0393
Epoch 120, Loss: 0.0355
Epoch 140, Loss: 0.0335
Epoch 160, Loss: 0.0317
Epoch 180, Loss: 0.0355
Test Loss: 0.0398
Test Accuracy: 0.9333


In [59]:
import numpy as np
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Activation functions
def identity(x):
    return x

def square(x):
    return np.power(x, 2)

def identity_derivative(x):
    return np.ones_like(x)

def square_derivative(x):
    return 2 * x

# Initialize network parameters
def initialize_parameters(input_size, hidden_size1, hidden_size2, hidden_size3, output_size):
    return {
        'W1': np.random.randn(input_size, hidden_size1) * 0.1,
        'b1': np.zeros((1, hidden_size1)),
        'W2': np.random.randn(input_size, hidden_size2) * 0.1,
        'b2': np.zeros((1, hidden_size2)),
        'W3': np.random.randn(hidden_size1 + hidden_size2, hidden_size3) * 0.1,
        'b3': np.zeros((1, hidden_size3)),
        'W4': np.random.randn(hidden_size3 + hidden_size1 + hidden_size2, output_size) * 0.1,
        'b4': np.zeros((1, output_size))
    }

# Forward pass
def forward_pass(X, params):
    Z1 = np.dot(X, params['W1']) + params['b1']
    A1 = identity(Z1)

    Z2 = np.dot(X, params['W2']) + params['b2']
    A2 = square(Z2)

    concatenated = np.concatenate((A1, A2), axis=1)

    Z3 = np.dot(concatenated, params['W3']) + params['b3']
    A3 = square(Z3)

    concatenated_A3 = np.concatenate((A3, concatenated), axis=1)

    Z4 = np.dot(concatenated_A3, params['W4']) + params['b4']
    A4 = identity(Z4)  # Linear activation

    return A4, (X, Z1, A1, Z2, A2, Z3, A3, concatenated_A3, Z4, A4, concatenated)

# Compute loss (Mean Squared Error)
def compute_loss(y_true, y_pred):
    return np.mean(np.square(y_true - y_pred))

# Backward pass
def backward_pass(y_true, cache, params):
    X, Z1, A1, Z2, A2, Z3, A3, concatenated_A3, Z4, A4, concatenated = cache

    dA4 = 2 * (A4 - y_true)
    dZ4 = dA4 * identity_derivative(Z4)  # Derivative of linear activation is 1
    dW4 = np.dot(concatenated_A3.T, dZ4)
    db4 = np.sum(dZ4, axis=0, keepdims=True)

    d_concatenated_A3 = np.dot(dZ4, params['W4'].T)
    dA3 = d_concatenated_A3[:, :hidden_size3]
    d_concatenated = d_concatenated_A3[:, hidden_size3:]

    dZ3 = dA3 * square_derivative(Z3)
    dW3 = np.dot(concatenated.T, dZ3)
    db3 = np.sum(dZ3, axis=0, keepdims=True)

    dA2 = d_concatenated[:, hidden_size1:]  # Corrected dA2 calculation
    d_concatenated_Z2 = dA2 * square_derivative(Z2)

    dA1 = d_concatenated[:, :hidden_size1]
    dZ1 = dA1 * identity_derivative(Z1)
    dW1 = np.dot(X.T, dZ1)
    db1 = np.sum(dZ1, axis=0, keepdims=True)

    dZ2 = d_concatenated_Z2
    dW2 = np.dot(X.T, dZ2)
    db2 = np.sum(dZ2, axis=0, keepdims=True)

    grads = {'dW1': dW1, 'db1': db1, 'dW2': dW2, 'db2': db2, 'dW3': dW3, 'db3': db3, 'dW4': dW4, 'db4': db4}
    return grads

# Update network parameters
def update_parameters(params, grads, learning_rate):
    for key in params.keys():
        params[key] -= learning_rate * grads['d' + key]
    return params

# Load and preprocess the Diabetes dataset
diabetes = load_diabetes()
X, y = diabetes.data, diabetes.target
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
Y = y.reshape(-1, 1)

# Split the dataset
X_train, X_test, Y_train, Y_test = train_test_split(X_scaled, Y, test_size=0.2, random_state=42)

# Network architecture
input_size = X_train.shape[1]
hidden_size1 = 10
hidden_size2 = 10
hidden_size3 = 10
output_size = 1

# Initialize parameters
params = initialize_parameters(input_size, hidden_size1, hidden_size2, hidden_size3, output_size)

# Training settings
epochs = 200
learning_rate = 0.001

# Training loop
for epoch in range(epochs):
    output, cache = forward_pass(X_train, params)
    loss = compute_loss(Y_train, output)
    grads = backward_pass(Y_train, cache, params)
    params = update_parameters(params, grads, learning_rate)

    if epoch % 20 == 0:
        print(f"Epoch {epoch}, Loss: {loss:.4f}")

# Evaluate the model
output_test, _ = forward_pass(X_test, params)
test_loss = compute_loss(Y_test, output_test)

print(f"Test Loss: {test_loss:.4f}")


Epoch 0, Loss: 29709.8532
Epoch 20, Loss: nan
Epoch 40, Loss: nan
Epoch 60, Loss: nan
Epoch 80, Loss: nan


/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:88: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


Epoch 100, Loss: nan
Epoch 120, Loss: nan
Epoch 140, Loss: nan
Epoch 160, Loss: nan
Epoch 180, Loss: nan
Test Loss: nan


In [70]:
import numpy as np
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Activation functions
def identity(x):
    return x

def square(x):
    return np.power(x, 2)/4

def identity_derivative(x):
    return np.ones_like(x)

def square_derivative(x):
    return  x/2


def square_2(x):
    return np.power(x, 2)/24

def square_2_derivative(x):
    return  x/12

# Initialize network parameters
def initialize_parameters(input_size, hidden_size1, hidden_size2, hidden_size3, output_size):
    return {
        'W1': np.random.randn(input_size, hidden_size1) * 0.1,
        'b1': np.zeros((1, hidden_size1)),
        'W2': np.random.randn(hidden_size1, hidden_size2) * 0.1,
        'b2': np.zeros((1, hidden_size2)),
        'W3': np.random.randn(hidden_size1 + hidden_size2, hidden_size3) * 0.1,
        'b3': np.zeros((1, hidden_size3)),
        'W4': np.random.randn(hidden_size3 + hidden_size1 + hidden_size2, output_size) * 0.1,
        'b4': np.zeros((1, output_size))
    }

# Forward pass
def forward_pass(X, params):
    Z1 = np.dot(X, params['W1']) + params['b1']
    A1 = identity(Z1)

    Z2 = np.dot(A1, params['W2']) + params['b2']
    A2 = square(Z2)

    concatenated = np.concatenate((A1, A2), axis=1)

    Z3 = np.dot(concatenated, params['W3']) + params['b3']
    A3 = square_2(Z3)

    concatenated_A3 = np.concatenate((A3, concatenated), axis=1)

    Z4 = np.dot(concatenated_A3, params['W4']) + params['b4']
    A4 = identity(Z4)  # Linear activation

    return A4, (X, Z1, A1, Z2, A2, Z3, A3, concatenated_A3, Z4, A4, concatenated)

# Compute loss (Mean Squared Error)
def compute_loss(y_true, y_pred):
    return np.mean(np.square(y_true - y_pred))

# Backward pass
def backward_pass(y_true, cache, params):
    X, Z1, A1, Z2, A2, Z3, A3, concatenated_A3, Z4, A4, concatenated = cache

    dA4 = 2 * (A4 - y_true)
    dZ4 = dA4 * identity_derivative(Z4)
    dW4 = np.dot(concatenated_A3.T, dZ4)
    db4 = np.sum(dZ4, axis=0, keepdims=True)

    d_concatenated_A3 = np.dot(dZ4, params['W4'].T)
    dA3 = d_concatenated_A3[:, :params['W3'].shape[1]]
    d_concatenated = d_concatenated_A3[:, params['W3'].shape[1]:]

    dZ3 = dA3 * square_derivative(Z3)
    dW3 = np.dot(concatenated.T, dZ3)
    db3 = np.sum(dZ3, axis=0, keepdims=True)

    dA2 = d_concatenated[:, hidden_size1:]  # Corrected dA2 calculation
    dZ2 = dA2 * square_derivative(Z2)
    dW2 = np.dot(A1.T, dZ2)
    db2 = np.sum(dZ2, axis=0, keepdims=True)

    dA1 = d_concatenated[:, :hidden_size1]
    dZ1 = dA1 * identity_derivative(Z1)
    dW1 = np.dot(X.T, dZ1)
    db1 = np.sum(dZ1, axis=0, keepdims=True)

    grads = {'dW1': dW1, 'db1': db1, 'dW2': dW2, 'db2': db2, 'dW3': dW3, 'db3': db3, 'dW4': dW4, 'db4': db4}
    return grads

# Update network parameters
def update_parameters(params, grads, learning_rate):
    for key in params.keys():
        params[key] -= learning_rate * grads['d' + key]
    return params

# Load and preprocess the Diabetes dataset
diabetes = load_diabetes()
X, y = diabetes.data, diabetes.target
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
Y = y.reshape(-1, 1)

# Split the dataset
X_train, X_test, Y_train, Y_test = train_test_split(X_scaled, Y, test_size=0.2, random_state=42)

# Network architecture
input_size = X_train.shape[1]
hidden_size1 = 10
hidden_size2 = 10
hidden_size3 = 10
output_size = 1

# Initialize parameters
params = initialize_parameters(input_size, hidden_size1, hidden_size2, hidden_size3, output_size)

# Training settings
epochs = 200
learning_rate = 0.001

# Training loop
for epoch in range(epochs):
    output, cache = forward_pass(X_train, params)
    loss = compute_loss(Y_train, output)
    grads = backward_pass(Y_train, cache, params)
    params = update_parameters(params, grads, learning_rate)

    if epoch % 20 == 0:
        print(f"Epoch {epoch}, Loss: {loss:.4f}")

# Evaluate the model
output_test, _ = forward_pass(X_test, params)
test_loss = compute_loss(Y_test, output_test)

print(f"Test Loss: {test_loss:.4f}")


Epoch 0, Loss: 29721.1327
Epoch 20, Loss: nan
Epoch 40, Loss: nan


<ipython-input-70-14750e32b4c0>:11: RuntimeWarning: overflow encountered in power
  return np.power(x, 2)


Epoch 60, Loss: nan
Epoch 80, Loss: nan
Epoch 100, Loss: nan
Epoch 120, Loss: nan
Epoch 140, Loss: nan
Epoch 160, Loss: nan
Epoch 180, Loss: nan
Test Loss: nan


In [71]:
import numpy as np
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Activation functions
def identity(x):
    return x

def square(x):
    return np.power(x, 2) / 4

def identity_derivative(x):
    return np.ones_like(x)

def square_derivative(x):
    return x / 2

def square_2(x):
    return np.power(x, 2) / 24

def square_2_derivative(x):
    return x / 12

# Initialize network parameters
def initialize_parameters(input_size, hidden_size1, hidden_size2, hidden_size3, output_size):
    return {
        'W1': np.random.randn(input_size, hidden_size1) * 0.1,
        'b1': np.zeros((1, hidden_size1)),
        'W2': np.random.randn(hidden_size1, hidden_size2) * 0.1,
        'b2': np.zeros((1, hidden_size2)),
        'W3': np.random.randn(hidden_size1 + hidden_size2, hidden_size3) * 0.1,
        'b3': np.zeros((1, hidden_size3)),
        'W4': np.random.randn(hidden_size3 + hidden_size1 + hidden_size2, output_size) * 0.1,
        'b4': np.zeros((1, output_size))
    }

# Forward pass
def forward_pass(X, params):
    Z1 = np.dot(X, params['W1']) + params['b1']
    A1 = identity(Z1)

    Z2 = np.dot(A1, params['W2']) + params['b2']
    A2 = square(Z2)

    concatenated = np.concatenate((A1, A2), axis=1)

    Z3 = np.dot(concatenated, params['W3']) + params['b3']
    A3 = square_2(Z3)

    concatenated_A3 = np.concatenate((A3, concatenated), axis=1)

    Z4 = np.dot(concatenated_A3, params['W4']) + params['b4']
    A4 = identity(Z4)  # Linear activation

    return A4, (X, Z1, A1, Z2, A2, Z3, A3, concatenated_A3, Z4, A4, concatenated)

# Compute loss (Mean Squared Error)
def compute_loss(y_true, y_pred):
    return np.mean(np.square(y_true - y_pred))

# Backward pass
def backward_pass(y_true, cache, params, hidden_size1, hidden_size2):
    X, Z1, A1, Z2, A2, Z3, A3, concatenated_A3, Z4, A4, concatenated = cache

    dA4 = 2 * (A4 - y_true)
    dZ4 = dA4 * identity_derivative(Z4)
    dW4 = np.dot(concatenated_A3.T, dZ4)
    db4 = np.sum(dZ4, axis=0, keepdims=True)

    d_concatenated_A3 = np.dot(dZ4, params['W4'].T)
    dA3 = d_concatenated_A3[:, :params['W3'].shape[1]]
    d_concatenated = d_concatenated_A3[:, params['W3'].shape[1]:]

    dZ3 = dA3 * square_2_derivative(Z3)
    dW3 = np.dot(concatenated.T, dZ3)
    db3 = np.sum(dZ3, axis=0, keepdims=True)

    dA2 = d_concatenated[:, hidden_size1:]
    dZ2 = dA2 * square_derivative(Z2)
    dW2 = np.dot(A1.T, dZ2)
    db2 = np.sum(dZ2, axis=0, keepdims=True)

    dA1 = d_concatenated[:, :hidden_size1]
    dZ1 = dA1 * identity_derivative(Z1)
    dW1 = np.dot(X.T, dZ1)
    db1 = np.sum(dZ1, axis=0, keepdims=True)

    grads = {'dW1': dW1, 'db1': db1, 'dW2': dW2, 'db2': db2, 'dW3': dW3, 'db3': db3, 'dW4': dW4, 'db4': db4}
    return grads

# Update network parameters
def update_parameters(params, grads, learning_rate):
    for key in params.keys():
        params[key] -= learning_rate * grads['d' + key]
    return params

# Load and preprocess the Diabetes dataset
diabetes = load_diabetes()
X, y = diabetes.data, diabetes.target
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
Y = y.reshape(-1, 1)

# Split the dataset
X_train, X_test, Y_train, Y_test = train_test_split(X_scaled, Y, test_size=0.2, random_state=42)

# Network architecture
input_size = X_train.shape[1]
hidden_size1 = 10
hidden_size2 = 10
hidden_size3 = 10
output_size = 1

# Initialize parameters
params = initialize_parameters(input_size, hidden_size1, hidden_size2, hidden_size3, output_size)

# Training settings
epochs = 200
learning_rate = 0.001

# Training loop
for epoch in range(epochs):
    output, cache = forward_pass(X_train, params)
    loss = compute_loss(Y_train, output)
    grads = backward_pass(Y_train, cache, params, hidden_size1, hidden_size2)
    params = update_parameters(params, grads, learning_rate)

    if epoch % 20 == 0:
        print(f"Epoch {epoch}, Loss: {loss:.4f}")

# Evaluate the model
output_test, _ = forward_pass(X_test, params)
test_loss = compute_loss(Y_test, output_test)

print(f"Test Loss: {test_loss:.4f}")


Epoch 0, Loss: 29728.5256
Epoch 20, Loss: nan


<ipython-input-71-66be0136e3cf>:11: RuntimeWarning: overflow encountered in power
  return np.power(x, 2) / 4
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:88: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


Epoch 40, Loss: nan
Epoch 60, Loss: nan
Epoch 80, Loss: nan
Epoch 100, Loss: nan
Epoch 120, Loss: nan
Epoch 140, Loss: nan
Epoch 160, Loss: nan
Epoch 180, Loss: nan
Test Loss: nan


In [77]:
import numpy as np
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# Activation functions
def identity(x):
    return x/2

def square(x):
    return np.power(x, 2) / 4

def identity_derivative(x):
    return np.ones_like(x)

def square_derivative(x):
    return x / 2

def square_2(x):
    return np.power(x, 2) / 24

def square_2_derivative(x):
    return x / 12

# Initialize network parameters
def initialize_parameters(input_size, hidden_size1, hidden_size2, hidden_size3, output_size):
    return {
        'W1': np.random.randn(input_size, hidden_size1) * 0.1,
        'b1': np.zeros((1, hidden_size1)),
        'W2': np.random.randn(hidden_size1, hidden_size2) * 0.1,
        'b2': np.zeros((1, hidden_size2)),
        'W3': np.random.randn(hidden_size1 + hidden_size2, hidden_size3) * 0.1,
        'b3': np.zeros((1, hidden_size3)),
        'W4': np.random.randn(hidden_size3 + hidden_size1 + hidden_size2, output_size) * 0.1,
        'b4': np.zeros((1, output_size))
    }

# Forward pass
def forward_pass(X, params):
    Z1 = np.dot(X, params['W1']) + params['b1']
    A1 = identity(Z1)

    Z2 = np.dot(A1, params['W2']) + params['b2']
    A2 = square(Z2)

    concatenated = np.concatenate((A1, A2), axis=1)

    Z3 = np.dot(concatenated, params['W3']) + params['b3']
    A3 = square_2(Z3)

    concatenated_A3 = np.concatenate((A3, concatenated), axis=1)

    Z4 = np.dot(concatenated_A3, params['W4']) + params['b4']
    A4 = identity(Z4)  # Linear activation

    return A4, (X, Z1, A1, Z2, A2, Z3, A3, concatenated_A3, Z4, A4, concatenated)

# Compute loss (Mean Squared Error)
def compute_loss(y_true, y_pred):
    return np.mean(np.square(y_true - y_pred))

# Backward pass
def backward_pass(y_true, cache, params, hidden_size1, hidden_size2):
    X, Z1, A1, Z2, A2, Z3, A3, concatenated_A3, Z4, A4, concatenated = cache

    dA4 = 2 * (A4 - y_true)
    dZ4 = dA4 * identity_derivative(Z4)
    dW4 = np.dot(concatenated_A3.T, dZ4)
    db4 = np.sum(dZ4, axis=0, keepdims=True)

    d_concatenated_A3 = np.dot(dZ4, params['W4'].T)
    dA3 = d_concatenated_A3[:, :params['W3'].shape[1]]
    d_concatenated = d_concatenated_A3[:, params['W3'].shape[1]:]

    dZ3 = dA3 * square_2_derivative(Z3)
    dW3 = np.dot(concatenated.T, dZ3)
    db3 = np.sum(dZ3, axis=0, keepdims=True)

    dA2 = d_concatenated[:, hidden_size1:]
    dZ2 = dA2 * square_derivative(Z2)
    dW2 = np.dot(A1.T, dZ2)
    db2 = np.sum(dZ2, axis=0, keepdims=True)

    dA1 = d_concatenated[:, :hidden_size1]
    dZ1 = dA1 * identity_derivative(Z1)
    dW1 = np.dot(X.T, dZ1)
    db1 = np.sum(dZ1, axis=0, keepdims=True)

    grads = {'dW1': dW1, 'db1': db1, 'dW2': dW2, 'db2': db2, 'dW3': dW3, 'db3': db3, 'dW4': dW4, 'db4': db4}
    return grads

# Update network parameters
def update_parameters(params, grads, learning_rate):
    for key in params.keys():
        params[key] -= learning_rate * grads['d' + key]
    return params

# Load and preprocess the Diabetes dataset
diabetes = load_diabetes()
X, y = diabetes.data, diabetes.target

# Apply MinMaxScaler to transform X to be in the range [0, 1]
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)
Y = y.reshape(-1, 1)

# Split the dataset
X_train, X_test, Y_train, Y_test = train_test_split(X_scaled, Y, test_size=0.2, random_state=42)

# Network architecture
input_size = X_train.shape[1]
hidden_size1 = 10
hidden_size2 = 10
hidden_size3 = 10
output_size = 1

# Initialize parameters
params = initialize_parameters(input_size, hidden_size1, hidden_size2, hidden_size3, output_size)

# Training settings
epochs = 200
learning_rate = 0.0001

# Training loop
for epoch in range(epochs):
    output, cache = forward_pass(X_train, params)
    loss = compute_loss(Y_train, output)
    grads = backward_pass(Y_train, cache, params, hidden_size1, hidden_size2)
    params = update_parameters(params, grads, learning_rate)

    if epoch % 20 == 0:
        print(f"Epoch {epoch}, Loss: {loss:.4f}")

# Evaluate the model
output_test, _ = forward_pass(X_test, params)
test_loss = compute_loss(Y_test, output_test)

print(f"Test Loss: {test_loss:.4f}")


Epoch 0, Loss: 29711.0636
Epoch 20, Loss: nan
Epoch 40, Loss: nan


<ipython-input-77-006922dac669>:20: RuntimeWarning: overflow encountered in power
  return np.power(x, 2) / 24


Epoch 60, Loss: nan
Epoch 80, Loss: nan
Epoch 100, Loss: nan
Epoch 120, Loss: nan
Epoch 140, Loss: nan
Epoch 160, Loss: nan
Epoch 180, Loss: nan
Test Loss: nan


In [79]:
import numpy as np
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# Activation functions
def identity(x):
    return x

def square(x):
    return np.power(x, 2) / 4

def identity_derivative(x):
    return np.ones_like(x)

def square_derivative(x):
    return x / 2

def square_2(x):
    return np.power(x, 2) / 24

def square_2_derivative(x):
    return x / 12

# Initialize network parameters
def initialize_parameters(input_size, hidden_size1, hidden_size2, hidden_size3, output_size):
    np.random.seed(42)  # For reproducibility
    return {
        'W1': np.random.randn(input_size, hidden_size1) * np.sqrt(2. / input_size),
        'b1': np.zeros((1, hidden_size1)),
        'W2': np.random.randn(hidden_size1, hidden_size2) * np.sqrt(2. / hidden_size1),
        'b2': np.zeros((1, hidden_size2)),
        'W3': np.random.randn(hidden_size1 + hidden_size2, hidden_size3) * np.sqrt(2. / (hidden_size1 + hidden_size2)),
        'b3': np.zeros((1, hidden_size3)),
        'W4': np.random.randn(hidden_size3 + hidden_size1 + hidden_size2, output_size) * np.sqrt(2. / (hidden_size3 + hidden_size1 + hidden_size2)),
        'b4': np.zeros((1, output_size))
    }

# Forward pass
def forward_pass(X, params):
    Z1 = np.dot(X, params['W1']) + params['b1']
    A1 = identity(Z1)

    Z2 = np.dot(A1, params['W2']) + params['b2']
    A2 = square(Z2)

    concatenated = np.concatenate((A1, A2), axis=1)

    Z3 = np.dot(concatenated, params['W3']) + params['b3']
    A3 = square_2(Z3)

    concatenated_A3 = np.concatenate((A3, concatenated), axis=1)

    Z4 = np.dot(concatenated_A3, params['W4']) + params['b4']
    A4 = identity(Z4)

    return A4, (X, Z1, A1, Z2, A2, Z3, A3, concatenated_A3, Z4, A4, concatenated)

# Compute loss (Mean Squared Error)
def compute_loss(y_true, y_pred):
    return np.mean(np.square(y_true - y_pred))

# Backward pass
def backward_pass(y_true, cache, params, hidden_size1, hidden_size2):
    X, Z1, A1, Z2, A2, Z3, A3, concatenated_A3, Z4, A4, concatenated = cache

    dA4 = 2 * (A4 - y_true)
    dZ4 = dA4 * identity_derivative(Z4)
    dW4 = np.dot(concatenated_A3.T, dZ4)
    db4 = np.sum(dZ4, axis=0, keepdims=True)

    d_concatenated_A3 = np.dot(dZ4, params['W4'].T)
    dA3 = d_concatenated_A3[:, :params['W3'].shape[1]]
    d_concatenated = d_concatenated_A3[:, params['W3'].shape[1]:]

    dZ3 = dA3 * square_2_derivative(Z3)
    dW3 = np.dot(concatenated.T, dZ3)
    db3 = np.sum(dZ3, axis=0, keepdims=True)

    dA2 = d_concatenated[:, hidden_size1:]
    dZ2 = dA2 * square_derivative(Z2)
    dW2 = np.dot(A1.T, dZ2)
    db2 = np.sum(dZ2, axis=0, keepdims=True)

    dA1 = d_concatenated[:, :hidden_size1]
    dZ1 = dA1 * identity_derivative(Z1)
    dW1 = np.dot(X.T, dZ1)
    db1 = np.sum(dZ1, axis=0, keepdims=True)

    grads = {'dW1': dW1, 'db1': db1, 'dW2': dW2, 'db2': db2, 'dW3': dW3, 'db3': db3, 'dW4': dW4, 'db4': db4}
    return grads

# Update network parameters
def update_parameters(params, grads, learning_rate):
    for key in params.keys():
        params[key] -= learning_rate * grads['d' + key]
    return params

# Load and preprocess the Diabetes dataset
diabetes = load_diabetes()
X, y = diabetes.data, diabetes.target

# Apply MinMaxScaler to transform X to be in the range [0, 1]
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)
Y = y.reshape(-1, 1)

# Split the dataset
X_train, X_test, Y_train, Y_test = train_test_split(X_scaled, Y, test_size=0.2, random_state=42)

# Network architecture
input_size = X_train.shape[1]
hidden_size1 = 10
hidden_size2 = 10
hidden_size3 = 10
output_size = 1

# Initialize parameters
params = initialize_parameters(input_size, hidden_size1, hidden_size2, hidden_size3, output_size)

# Training settings
epochs = 200
learning_rate = 0.00001  # Adjusted learning rate

# Training loop
for epoch in range(epochs):
    output, cache = forward_pass(X_train, params)
    loss = compute_loss(Y_train, output)
    grads = backward_pass(Y_train, cache, params, hidden_size1, hidden_size2)
    params = update_parameters(params, grads, learning_rate)

    if epoch % 20 == 0:
        print(f"Epoch {epoch}, Loss: {loss:.4f}")

# Evaluate the model
output_test, _ = forward_pass(X_test, params)
test_loss = compute_loss(Y_test, output_test)

print(f"Test Loss: {test_loss:.4f}")


Epoch 0, Loss: 29889.3644
Epoch 20, Loss: nan
Epoch 40, Loss: nan
Epoch 60, Loss: nan
Epoch 80, Loss: nan
Epoch 100, Loss: nan
Epoch 120, Loss: nan
Epoch 140, Loss: nan
Epoch 160, Loss: nan
Epoch 180, Loss: nan
Test Loss: nan


In [82]:
import numpy as np
import tensorflow as tf
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Model
from keras.layers import Input, Dense, Concatenate, Lambda

# Custom activation function using TensorFlow operations
def custom_square_activation(x):
    return tf.math.pow(x, 2) / 4

def custom_square_2_activation(x):
    return tf.math.pow(x, 2) / 24

# Load Iris dataset
iris_data = load_iris()
X, y = iris_data.data, iris_data.target.reshape(-1, 1)

# Preprocess the data: Normalize X
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Define the input size and hidden sizes
input_size = X_scaled.shape[1]
hidden_size1 = 10
hidden_size2 = 10
hidden_size3 = 10
output_size = 1  # For regression, it's 1; for classification, it should match the number of classes

# Define the model architecture
input_layer = Input(shape=(input_size,))

hidden_layer_1 = Dense(hidden_size1, activation='linear')(input_layer)
hidden_layer_2 = Dense(hidden_size2)(input_layer)
hidden_layer_2_activated = Lambda(custom_square_activation)(hidden_layer_2)

concatenated_layer_1 = Concatenate()([hidden_layer_1, hidden_layer_2_activated])

hidden_layer_3 = Dense(hidden_size3)(concatenated_layer_1)
hidden_layer_3_activated = Lambda(custom_square_2_activation)(hidden_layer_3)

concatenated_layer_2 = Concatenate()([hidden_layer_3_activated, concatenated_layer_1])

output_layer = Dense(output_size, activation='linear')(concatenated_layer_2)

model = Model(inputs=input_layer, outputs=output_layer)
model.compile(optimizer='adam', loss='mean_squared_error')

# Print model summary
model.summary()

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Train the model
history = model.fit(X_train, y_train, epochs=100, batch_size=10, validation_split=0.2)

# Evaluate the model on the test set
test_loss = model.evaluate(X_test, y_test)
print(f'Test loss: {test_loss}')


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, 4)]                  0         []                            
                                                                                                  
 dense_19 (Dense)            (None, 10)                   50        ['input_3[0][0]']             
                                                                                                  
 dense_18 (Dense)            (None, 10)                   50        ['input_3[0][0]']             
                                                                                                  
 lambda_2 (Lambda)           (None, 10)                   0         ['dense_19[0][0]']            
                                                                                            

In [83]:
import numpy as np
import tensorflow as tf
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from keras.models import Model
from keras.layers import Input, Dense, Concatenate, Lambda
from keras.utils import to_categorical

# Custom activation function using TensorFlow operations
def custom_square_activation(x):
    return tf.math.pow(x, 2) / 4

def custom_square_2_activation(x):
    return tf.math.pow(x, 2) / 24

# Load Iris dataset
iris_data = load_iris()
X, y = iris_data.data, iris_data.target

# Preprocess the data: Normalize X
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# One-hot encode y
y_encoded = to_categorical(y)

# Define the input size and hidden sizes
input_size = X_scaled.shape[1]
hidden_size1 = 10
hidden_size2 = 10
hidden_size3 = 10
output_size = 3  # This should match the number of classes for classification

# Define the model architecture
input_layer = Input(shape=(input_size,))

hidden_layer_1 = Dense(hidden_size1, activation='linear')(input_layer)
hidden_layer_2 = Dense(hidden_size2)(input_layer)
hidden_layer_2_activated = Lambda(custom_square_activation)(hidden_layer_2)

concatenated_layer_1 = Concatenate()([hidden_layer_1, hidden_layer_2_activated])

hidden_layer_3 = Dense(hidden_size3)(concatenated_layer_1)
hidden_layer_3_activated = Lambda(custom_square_2_activation)(hidden_layer_3)

concatenated_layer_2 = Concatenate()([hidden_layer_3_activated, concatenated_layer_1])

output_layer = Dense(output_size, activation='softmax')(concatenated_layer_2)

model = Model(inputs=input_layer, outputs=output_layer)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print model summary
model.summary()

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_encoded, test_size=0.2, random_state=42)

# Train the model
history = model.fit(X_train, y_train, epochs=100, batch_size=10, validation_split=0.2)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f'Test loss: {test_loss}')
print(f'Test accuracy: {test_accuracy}')


Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_4 (InputLayer)        [(None, 4)]                  0         []                            
                                                                                                  
 dense_23 (Dense)            (None, 10)                   50        ['input_4[0][0]']             
                                                                                                  
 dense_22 (Dense)            (None, 10)                   50        ['input_4[0][0]']             
                                                                                                  
 lambda_4 (Lambda)           (None, 10)                   0         ['dense_23[0][0]']            
                                                                                            